# Validando a Clusterização do Gemini

Validando via código os resultados da Clusterização feita pelo Gemini em dados de rotatividade de funcionários cujos resultados foram compartilhados no meu [artigo publicado no Medium](https://medium.com/@luizalaquini/ci%C3%AAncia-de-dados-sem-c%C3%B3digo-um-exemplo-de-clusteriza%C3%A7%C3%A3o-por-ia-generativa-dee5f7224cc2).

**Por:** Luiza Batista Laquini